# Limitation(s) of sklearn’s NMF Library 

DTSA 5510 - Unsupervised Algorithms in Machine Learning

University of Colorado Boulder

## Part 1
### Import Modules

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from collections import namedtuple
from scipy.sparse import csr_matrix
from sklearn.decomposition import NMF
import time

### Load Data

Loading the same MovieLens 1M dataset from the homework. Luckily, it was hosted here on kaggle.

In [ ]:
users_df = pd.read_csv('/kaggle/input/movielens-1m-dataset/users.dat',
                       header=None, 
                       sep='::', 
                       names=['UserID','Gender','Age','Occupation','Zip-code'], 
                       engine='python',
                       encoding='latin-1')


movies_df = pd.read_csv('/kaggle/input/movielens-1m-dataset/movies.dat',
                        header=None,
                        sep='::',
                        names=['MovieID', 'Title', 'Genre'], 
                        engine='python',
                        encoding='latin-1')

ratings_df = pd.read_csv('/kaggle/input/movielens-1m-dataset/ratings.dat',
                         header=None,
                         sep='::',
                         names=['UserID','MovieID','Rating','Timestamp'], 
                         engine='python',
                         encoding='latin-1')

Below, I split the ratings data into a train subset (80%) and a test subset (20%). Then, I put it all together into a names tuple for convenience and print the first few rows of each data subset.

In [ ]:
train_df, test_df = train_test_split(ratings_df, test_size=0.2, random_state=19)

Data = namedtuple('Data', ['users','movies','train','test'])
data = Data(users_df, movies_df, train_df, test_df)

print('\n',data.users.head())
print('\n',data.movies.head())
print('\n',data.train.head())
print('\n',data.test.head())

### NMF Matrix Factorization Custom Implementation

Initialized like the RecSys class from the original homework notebook. Has methods for building the rating matrix, fitting NMF, making predictions, and scoreing using RMSE.

In [54]:
class MatrixFactorization():
    
    def __init__(self, data):
            self.data=data
            self.allusers = list(self.data.users['UserID'])
            self.allmovies = list(self.data.movies['MovieID'])
            self.mid2idx = dict(zip(self.data.movies.MovieID,list(range(len(self.data.movies)))))
            self.uid2idx = dict(zip(self.data.users.UserID,list(range(len(self.data.users)))))
            self.Mr=self.rating_matrix()
            self.W = None  # User factors
            self.H = None  # Item factors
            self.reconstructed = None  # Reconstructed matrix
            self.n_components_ = None

    def rating_matrix(self):

        ind_movie = [self.mid2idx[x] for x in self.data.train.MovieID] 
        ind_user = [self.uid2idx[x] for x in self.data.train.UserID]
        rating_train = list(self.data.train.Rating)
        
        return csr_matrix((rating_train, (ind_user, ind_movie)), shape=(len(self.allusers), len(self.allmovies)))

    
    def fit_nmf(self, n_components=1, max_iter=200, beta_loss='frobenius', init='random', solver='cd'):
        
        Mr_dense = self.Mr.toarray()
        
        global_mean = Mr_dense[Mr_dense > 0].mean()
        
        # impute with global mean 
        Mr_filled = Mr_dense.copy()
        Mr_filled[Mr_filled == 0] = global_mean
        
        # initialize and fit NMF model            
        model = NMF(n_components=n_components, max_iter=max_iter, beta_loss=beta_loss, 
                    init=init, solver=solver,random_state=19)
        
        start_time = time.time()
        self.W = model.fit_transform(Mr_filled)
        self.H = model.components_
        self.n_components_ = model.n_components_
        
        # rebuild the ratings matrix
        self.reconstructed = np.dot(self.W, self.H)
        
        elapsed = time.time() - start_time
        print(f"NMF fitting time {elapsed}")
        
        return self
    
    def predict_from_nmf(self, uid, mid):

        if self.reconstructed is None:
            return 3.0  
        
        user_idx = self.uid2idx[uid]
        movie_idx = self.mid2idx[mid]
        
        # clip to range (1-5)
        prediction = self.reconstructed[user_idx, movie_idx]
        return np.clip(prediction, 1, 5)
    
    def predict(self):

        uids = self.data.test['UserID'].values
        mids = self.data.test['MovieID'].values
        
        predictions = np.array([self.predict_from_nmf(uid, mid) for uid, mid in zip(uids, mids)])
        
        return predictions
        
    def rmse(self,yp):
        yp[np.isnan(yp)]=3 #In case there is nan values in prediction, it will impute to 3.
        yt=np.array(self.data.test.Rating)
        return np.sqrt(((yt-yp)**2).mean())

In [55]:
for k in [1, 50, 100, 150]:

        mf = MatrixFactorization(data)

        mf.fit_nmf(n_components=k)

        predictions = mf.predict()
        rmse = mf.rmse(predictions)

        print(f"NMF with {mf.n_components_} components RMSE: {rmse}")

NMF fitting time 0.3909461498260498
NMF with 1 components RMSE: 1.1737815382990575


/usr/local/lib/python3.10/dist-packages/sklearn/decomposition/_nmf.py:1665: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve convergence.
  warnings.warn(


NMF fitting time 28.286161184310913
NMF with 50 components RMSE: 1.0726620197231886


/usr/local/lib/python3.10/dist-packages/sklearn/decomposition/_nmf.py:1665: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve convergence.
  warnings.warn(


NMF fitting time 62.16630959510803
NMF with 100 components RMSE: 1.0983095865459571


/usr/local/lib/python3.10/dist-packages/sklearn/decomposition/_nmf.py:1665: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve convergence.
  warnings.warn(


NMF fitting time 117.34989380836487
NMF with 150 components RMSE: 1.119522350478055


## Part 2

The NMF approach yielded high RMSE values. The best RMSE achieved was 1.07, which is within the range of values achieved with similarity based methods, but not particularly impressive. The table below shows each method used and their RMSE.

|Method|RMSE|
|:----|:--------:|
|Baseline, $Y_p$=3|1.26 |
|Baseline, $Y_p=\mu_u$|1.14 |
|Content based, item-item|1.19 |
|Collaborative, cosine|1.14 |
|Collaborative, jaccard, $M_r\geq 3$|0.98  |
|Collaborative, jaccard, $M_r\geq 1$|0.99  |
|Collaborative, jaccard, $M_r$|0.96|

It is clear that the matrix factorization approach (RMSE 1.07) is much better than the baseline methods that imputed values with either the median rating (RMSE 1.26) or the mean rating (RMSE 1.14). Non-negative Matrix Factorization is also superior to the content based item-item (RMSE 1.19) and collaborative appraoch with cosine similarity (RMSE 1.14) methods. 

NMF falls short compared to collaboritve recommendation using jaccard similarity in general. Three strategies were used to construct the ratings (utility) matrix for the collaborative recommendation methods. The first was to use only the good ratings (>2 on a 1-5 scale), and this achieved an RMSE of 0.98. The second was to use only the existing ratings (>0), which achieved an RMSE of 0.99. The third was to use the data without applying a transformation to achieve the lowest RMSE of 0.96.

The inferior performance can likely be attributed to NMF's inability to handle sparse data. The other recommendation methods can take advantage of sparse matrices, but NMF requires dense matrices and imputation. Also, the better-performing jaccard-based methods transform contuous ratings into binary, which may allow this approach to capture preference patterns more accurately than the continuous ratings used by NMF and the rest.

Overall, NMF is too generic to compete with the more specialized recommendation systems. To improve the error, it may be beneficial to transform the utlity matrix to contain binary values as seen in the jaccard-based methods. Additionally, optimizing the hyperparameters can likely improve the capacity to recognize preference patterns.
